So I just ran the find_SARFI.py script for the first time. It ran overnight but kept throwing errors like:

In [ ]:
# ... terminal ends
# Row 194919 couldn't find a stamp file :/
# ... many lines of exactly the same thing
# Row 196081 couldn't find a stamp file :/
# Row 196082 couldn't find a stamp file :/
# Row 196083 couldn't find a stamp file :/
# Row 196084 couldn't find a stamp file :/
# Row 196085 couldn't find a stamp file :/
# Row 196086 couldn't find a stamp file :/
# Row 196087 couldn't find a stamp file :/
# Row 196088 couldn't find a stamp file :/
# Row 196089 couldn't find a stamp file :/
# Row 196090 couldn't find a stamp file :/
# Row 196091 couldn't find a stamp file :/
# Row 196093 couldn't find a stamp file :/
# Row 196094 couldn't find a stamp file :/
# Row 196095 couldn't find a stamp file :/
# Row 196096 couldn't find a stamp file :/
# Row 196097 couldn't find a stamp file :/
# Row 196099 couldn't find a stamp file :/
# Row 196100 couldn't find a stamp file :/
# Row 196101 couldn't find a stamp file :/
# Row 196102 couldn't find a stamp file :/
# Row 196103 couldn't find a stamp file :/
# Row 196104 couldn't find a stamp file :/

Then it finished with the error:

In [ ]:
# Traceback (most recent call last):
#   File "find_SARFI.py", line 166, in <module>
#     stamp = find_stamp_of_hit(row.file_uri, row.signal_frequency)
#   File "find_SARFI.py", line 120, in find_stamp_of_hit
#     for stamp in stamps_gen:
#   File "/home/nstiegle/miniforge3/envs/stamp_viewer/lib/python3.7/site-packages/seticore/viewer.py", line 684, in read_stamps
#     with open(filepath) as f:
# FileNotFoundError: [Errno 2] No such file or directory: '/mnt/cosmic-storage-2/data2/vla_target/23B-169.sb44841813.eb44870447.60285.9953211574/23B-169.sb44841813.eb4487

So I want to investigate why it couldn't find the stamps in the stamp files for these rows and also why it couldn't find the stamp file for the row that crashed it

In [13]:
# Import packages needed
import numpy as np
import pandas as pd
import pickle
import os
import socket
import argparse
from datetime import datetime
from seticore import viewer # If you don't have it, use pip install "git+https://github.com/MydonSolutions/seticore#egg=seticore&subdirectory=python"
import matplotlib.pyplot as plt

In [2]:
# Read in hits data
df = pd.read_csv("/home/nstiegle/BL-COSMIC-2024-proj/stamps/representative_samples/1in25_targets.csv")

In [7]:
# Read in laest checkpoint
checkpoint_file = "/home/nstiegle/BL-COSMIC-2024-proj/stamps/representative_samples/1in25_targets_ckpts/1in25_targets_ckpt196000.npy"
checkpoint = np.load(checkpoint_file, allow_pickle=True)
all_snrs = checkpoint[:, 0]
all_signals = checkpoint[:, 1]

In [35]:
# Figure out how far it got
# Figure out what number to start at
def find_first_none(array):
    for i, item in enumerate(array):
        if type(item) == type(None):
            return i

first_none_snrs = find_first_none(all_snrs)
first_none_signals = find_first_none(all_signals)
assert(all_snrs[first_none_snrs] == None)
assert(all_signals[first_none_signals] == None)
assert(first_none_snrs == first_none_signals)
first_none = first_none_snrs
print(f"The checkpoint is from 196000, so it makes sense that everything after that isn't assigned yet (None)")
last_good = first_none - 1

The checkpoint is from 196000, so it makes sense that everything after that isn't assigned yet (None)


In [ ]:
# This makes sense from looking at the errors around 196001:
#
# Row 195999 couldn't find a stamp file :/
# On target 196000 at 2024-07-25 09:18:24.921711. Saving progress
# Done intermediate save for target 196000 at 2024-07-25 09:18:25.895863
# Row 196002 couldn't find a stamp file :/ 
#
# Notice how there's an error for 196599 and 196002 but not 196000 or 196001
# it's just that 196001 didn't get saved

In [47]:
# So a better question is how many are [] (rows that couldn't find stamps)
# and why that's the case
snrs_are_empty = np.array([type(entry) == type([]) for entry in all_snrs[:first_none]])
signals_are_empty = np.array([type(entry) == type([]) for entry in all_signals[:first_none]])
assert(np.array_equal(snrs_are_empty, signals_are_empty)) # So if one is empty the other is empty as expected
num_entries = len(snrs_are_empty)
num_empty = sum(snrs_are_empty)
num_full =  num_entries - num_empty
print(f"Of {num_entries} run antenna signals, {num_empty} entries are [] (leaving {num_full} actual values, {round(num_full / num_entries * 100, 3)}%)")

Of 196001 run antenna signals, 132439 entries are [] (leaving 63562 actual values, 32.429%)


In [102]:
# Let's investigate the first time it gives us a []
print("Investigating the first row with a []")
index_of_first_empty = np.where(signals_are_empty)[0][0]
first_empty_row = df.iloc[index_of_first_empty]
row_before_first_empty_row = df.iloc[index_of_first_empty - 1]
print("source", first_empty_row.source_name)
print("frequency", first_empty_row.signal_frequency, "MHz")

Investigating the first row with a []
source 3127348761205770496
frequency 42279.70805240249 MHz


In [75]:
# Looking at things which changed from the last row to this row
print("What changed from the last row to this row?")
print("The items which changed from the last row to this row are", first_empty_row.index[~(first_empty_row == row_before_first_empty_row)])

# Print the useful part of the file path
def print_path_end(file_uri):
    return ".".join(file_uri.split("/")[-1].split(".")[-6:-4])

# Compare things bewteen rows
def compare_rows(rows, col_name):
    print("Comparing", col_name)
    # Print special for URI so it's not quite as long
    if col_name == "file_uri":
        for row in rows:
            print(row, print_path_end(df.iloc[row][col_name]))
    else:
        for row in rows:
            print(row, df.iloc[row][col_name])
    print()

compare_rows([index_of_first_empty + 1, index_of_first_empty, index_of_first_empty - 1], "file_uri")
compare_rows([index_of_first_empty + 1, index_of_first_empty, index_of_first_empty - 1], "source_name")
compare_rows([index_of_first_empty + 1, index_of_first_empty, index_of_first_empty - 1], "ra_hours")
compare_rows([index_of_first_empty + 1, index_of_first_empty, index_of_first_empty - 1], "dec_degrees")

print("So it's right when it switches targets")

What changed from the last row to this row?
The items which changed from the last row to this row are Index(['Unnamed: 0', 'id', 'beam_id', 'subband_offset', 'file_uri',
       'file_local_enumeration', 'signal_frequency', 'signal_index',
       'signal_snr', 'signal_coarse_channel', 'signal_beam', 'signal_power',
       'signal_incoherent_power', 'source_name', 'fch1_mhz', 'ra_hours',
       'dec_degrees', 'coarse_channel', 'start_channel'],
      dtype='object')
Comparing file_uri
816 BD.C704
815 BD.C0
814 BD.C416

Comparing source_name
816 3127348761205770496
815 3127348761205770496
814 Incoherent

Comparing ra_hours
816 6.789733333333333
815 6.789733333333333
814 6.789705555628406

Comparing dec_degrees
816 3.27776
815 3.27776
814 3.279072222883132



In [108]:
# What's in that file?
print("We're going to investigate what is in the stamps file for the one it couldn't find the hit of")

# Convert a filepath from pointing to a .hits
# to a .stamps file for the same uri
# If given a .stamps ending, doesn't change it
def stamp_filepath_of(hits_filepath):
    return hits_filepath.replace('.hits', '.0000.stamps')

assert(os.path.exists(stamp_filepath_of(first_empty_row.file_uri)))
stamps_gen = viewer.read_stamps(stamp_filepath_of(first_empty_row.file_uri))
stamps_frequencies = np.array([stamp.stamp.signal.frequency for stamp in stamps_gen])
print("There are", stamps_frequencies.shape, "stamps in the file")
print("The minimum difference between the frequency of the hit and the frequency of the stamp is", abs(stamps_frequencies - first_empty_row.signal_frequency).min(), "MHz")
print("Did the hit point to the right file?")
print("Well there are 2473 hits which have the same URI")

We're going to investigate what is in the stamps file for the one it couldn't find the hit of
There are (500,) stamps in the file
The minimum difference between the frequency of the hit and the frequency of the stamp is 25.38838195801509 MHz
Did the hit point to the right file?
Well there are 2473 hits which have the same URI


In [ ]:
# And this is what the folder looks like. Turns out there can be multiple stamps files if there are too many to all fit in one
# I'll adjust the routine to look for the next stamp file if it doesn't find the stamps in the right one
#
# -rw-r--r--  1 cosmic cosmic  130K Sep 25  2023 23A-362.sb43789194.eb44583327.60212.544836180554.23.1.BD.C0.0000.bfr5
# -rw-r--r--  1 cosmic cosmic   13K Sep 25  2023 23A-362.sb43789194.eb44583327.60212.544836180554.23.1.BD.C0.0000.raw.blade.stdout.txt
# -rw-r--r--  1 cosmic cosmic  1.4G Sep 25  2023 23A-362.sb43789194.eb44583327.60212.544836180554.23.1.BD.C0.0000.raw.seticore.0000.stamps
# -rw-r--r--  1 cosmic cosmic  1.4G Sep 25  2023 23A-362.sb43789194.eb44583327.60212.544836180554.23.1.BD.C0.0000.raw.seticore.0001.stamps
# -rw-r--r--  1 cosmic cosmic  1.4G Sep 25  2023 23A-362.sb43789194.eb44583327.60212.544836180554.23.1.BD.C0.0000.raw.seticore.0002.stamps
# -rw-r--r--  1 cosmic cosmic  1.4G Sep 25  2023 23A-362.sb43789194.eb44583327.60212.544836180554.23.1.BD.C0.0000.raw.seticore.0003.stamps
# -rw-r--r--  1 cosmic cosmic  492M Sep 25  2023 23A-362.sb43789194.eb44583327.60212.544836180554.23.1.BD.C0.0000.raw.seticore.0004.stamps
# -rw-r--r--  1 cosmic cosmic   13M Sep 25  2023 23A-362.sb43789194.eb44583327.60212.544836180554.23.1.BD.C0.0000.raw.seticore.hits

In [143]:
# Do the search on this stamp
def revised_find_stamp_of_hit(hit_uri, hit_frequency, threshold=500):
    # Look for stamp in the 0000 stamp file
    stamp = find_stamp_of_hit(hit_uri, hit_frequency, threshold)
    
    # Given a stamp_uri, increments the index of the stamp file by one (so if it's /.../...seticore.0000.stamps it goes to /.../...seticore.0001.stamps)
    def increment_stamp_uri(stamp_uri):
        split_uri = stamp_uri.split('.')
        assert(split_uri[-1] == 'stamps')
        num = int(split_uri[-2])
        num += 1
        split_uri[-2] = str(num).zfill(4)
        return ".".join(split_uri)
    
    # If not found in 0000 stamp file, look for it in the 0001 etc stamp files
    # until we either find it or run out of files
    stamp_uri = increment_stamp_uri(stamp_filepath_of(hit_uri))
    while (stamp == None) and (os.path.exists(stamp_uri)):
        print("Looking in", stamp_uri.split('.')[-2])
        stamp = find_stamp_of_hit(stamp_uri, hit_frequency, threshold)
        stamp_uri = increment_stamp_uri(stamp_uri)

    # If we've found the stamp or run out of files, return it
    return stamp

revised_find_stamp_of_hit(first_empty_row.file_uri, first_empty_row.signal_frequency)

/home/nstiegle/miniforge3/envs/stamp_viewer/lib/python3.7/site-packages/seticore/viewer.py:125: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.h5 = h5py.File(filepath)


Looking in 0001
Looking in 0002
Looking in 0003
Looking in 0004


In [118]:
# What happened with the one the script failed on?
# That would be row 196104 + 1 = 196105 I think
# Unless 105 worked and then it kept working till it failed
# Let's check
# Error was from file: '/mnt/cosmic-storage-2/data2/vla_target/23B-169.sb44841813.eb44870447.60285.9953211574/23B-169.sb44841813.eb4487
# But maybe it got cut off?
row = df.iloc[196105]
row.file_uri
assert(os.path.exists(row.file_uri))
assert(os.path.exists(stamp_filepath_of(row.file_uri)))
stamps_fs = np.array([stamp.stamp.signal.frequency for stamp in viewer.read_stamps(stamp_filepath_of(row.file_uri))])
print(stamps_fs.shape)
print((abs(stamps_fs - row.signal_frequency) < 1e-7).sum(), "signal within 1e-7 in the stamp file")


(500,)
1 signal within 1e-7 in the stamp file


In [121]:
# Do the search on this stamp
def find_stamp_of_hit(hit_uri, hit_frequency, threshold=500):
    stamp_uri = stamp_filepath_of(hit_uri)
    stamps_gen = viewer.read_stamps(stamp_uri, find_recipe=True)
    for stamp in stamps_gen:
        assert(stamp != None)
        assert(stamp.recipe != None)
        if abs(stamp.stamp.signal.frequency - hit_frequency) < threshold*1e-6:
            # Found the stamp!
            return stamp
    
    # Didn't find the stamp in this whole file of stamps
    return None

def antenna_signal_snr_power(stamp):
    # Get the powers in the frequency bins of each antenna by summing 
    # over polarization and complex magnitude
    # Also rearrange so indices are (antenna, time bin, frequency bin)
    antenna_powers = np.square(stamp.real_array()).sum(axis=(2, 4)).transpose(2, 0, 1)
    snr_and_signals = np.array([stamp.snr_and_signal(antenna_power) for antenna_power in antenna_powers])
    return (snr_and_signals[:, 0], snr_and_signals[:, 1])

stamp = find_stamp_of_hit(row.file_uri, row.signal_frequency, threshold=500)
assert(stamp != None)
snrs, signals = antenna_signal_snr_power(stamp)
assert(type(snrs) != type([]))
assert(type(signals) != type([]))

/home/nstiegle/miniforge3/envs/stamp_viewer/lib/python3.7/site-packages/seticore/viewer.py:125: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.h5 = h5py.File(filepath)


In [125]:
# I guess let's automate this and check forwards until something crashes
indices_with_errors = []
for i in range(196105, 197101):
    try:
        row = df.iloc[i]
        assert os.path.exists(row.file_uri), ".hits doesn't exist"
        assert os.path.exists(stamp_filepath_of(row.file_uri)), ".stamp doesn't exist"
        stamp = find_stamp_of_hit(row.file_uri, row.signal_frequency, threshold=500)
        assert stamp != None, "stamp is None" 
        snrs, signals = antenna_signal_snr_power(stamp)
        assert type(snrs) != type([]), "snrs is empty" 
        assert type(signals) != type([]), "signals is empty"
    except Exception as err:
        print("Was on", i)
        indices_with_errors.append(i)
        print(err)

/home/nstiegle/miniforge3/envs/stamp_viewer/lib/python3.7/site-packages/seticore/viewer.py:125: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.h5 = h5py.File(filepath)


Was on 196127
.hits doesn't exist
Was on 196145
.hits doesn't exist
Was on 196160
.hits doesn't exist
Was on 196189
.hits doesn't exist


In [133]:
print(indices_with_errors)
hits_files = np.array([df.iloc[index].file_uri for index in indices_with_errors])
print("There are", np.unique(hits_files).shape, "unique hits files which are causing issues")
bad_file = np.unique(hits_files)[0]
print(bad_file)

[196127, 196145, 196160, 196189]
There are (1,) unique hits files which are causing issues
/mnt/cosmic-storage-2/data2/vla_target/23B-169.sb44841813.eb44870447.60285.9953211574/23B-169.sb44841813.eb44870447.60285.9953211574.48.1/23B-169.sb44841813.eb44870447.60285.9953211574.48.1.AC.C800.0000.raw.seticore.hits


In [ ]:
# So I'll have to be careful for files that don't exist